***GENERATED CODE FOR cities PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'name', 'transformation_label': 'String Indexer'}], 'feature': 'name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '"""Elio Suite""  Vatican - Long term Available"',
                                                  'max': 'æ—\xa0æ•Œå¤©æ™¯52å±‚sky bar æ›¼è°·è±ªå\x8dŽå¤§åºŠæˆ¿circle 2ï¼Œæ—\xa0è¾¹æ³³æ±\xa0ï¼Œå\x81¥èº«æ¡‘æ‹¿ï¼Œäº¤é€šä¾¿åˆ©ï¼Œæ˜Žæ˜Ÿå…¬å¯“', 'missing': '0', 'distinct': '499'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'name'}, {'feature_label': 'name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_id', 'transformation_label': 'String Indexer'}], 'feature': 'host_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '121441927.14', 'stddev': '112299457.22', 'min': '100552972', 'max': '99487919', 'missing': '0', 'distinct': '393'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_id'}, {'feature_label': 'host_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_since', 'transformation_label': 'String Indexer'}], 'feature': 'host_since', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2013', 'max': '9/9/2020', 'missing': '0', 'distinct': '429'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_since'}, {'feature_label': 'host_since', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_since')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_response_time', 'transformation_label': 'String Indexer'}], 'feature': 'host_response_time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'a few days or more', 'max': 'within an hour', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_response_time'}, {'feature_label': 'host_response_time', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_response_time')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_response_rate', 'transformation_label': 'String Indexer'}], 'feature': 'host_response_rate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.79', 'stddev': '0.31', 'min': '0', 'max': '1', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_response_rate'}, {'feature_label': 'host_response_rate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_response_rate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_acceptance_rate', 'transformation_label': 'String Indexer'}], 'feature': 'host_acceptance_rate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.84', 'stddev': '0.25', 'min': '0', 'max': '1', 'missing': '0', 'distinct': '52'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_acceptance_rate'}, {'feature_label': 'host_acceptance_rate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_acceptance_rate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_is_superhost', 'transformation_label': 'String Indexer'}], 'feature': 'host_is_superhost', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_is_superhost'}, {'feature_label': 'host_is_superhost', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_is_superhost')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_total_listings_count', 'transformation_label': 'String Indexer'}], 'feature': 'host_total_listings_count', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': '1', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_total_listings_count'}, {'feature_label': 'host_total_listings_count', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'host_total_listings_count')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_has_profile_pic', 'transformation_label': 'String Indexer'}], 'feature': 'host_has_profile_pic', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_has_profile_pic'}, {'feature_label': 'host_has_profile_pic', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_has_profile_pic')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_identity_verified', 'transformation_label': 'String Indexer'}], 'feature': 'host_identity_verified', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_identity_verified'}, {'feature_label': 'host_identity_verified', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_identity_verified')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'neighbourhood', 'transformation_label': 'String Indexer'}], 'feature': 'neighbourhood', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Adalar', 'max': 'Yuen Long', 'missing': '0', 'distinct': '148'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'neighbourhood'}, {'feature_label': 'neighbourhood', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('neighbourhood')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bangkok', 'max': 'Sydney', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'latitude', 'transformation_label': 'String Indexer'}], 'feature': 'latitude', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '8.5', 'stddev': '31.27', 'min': '-22.87791', 'max': '48.89594', 'missing': '0', 'distinct': '489'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'latitude'}, {'feature_label': 'latitude', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('latitude')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'longitude', 'transformation_label': 'String Indexer'}], 'feature': 'longitude', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '13.17', 'stddev': '77.78', 'min': '-43.16027', 'max': '29.30742', 'missing': '0', 'distinct': '489'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'longitude'}, {'feature_label': 'longitude', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('longitude')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'property_type', 'transformation_label': 'String Indexer'}], 'feature': 'property_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Entire bungalow', 'max': 'Shared room in house', 'missing': '0', 'distinct': '38'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'property_type'}, {'feature_label': 'property_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('property_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'room_type', 'transformation_label': 'String Indexer'}], 'feature': 'room_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Entire place', 'max': 'Shared room', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'room_type'}, {'feature_label': 'room_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('room_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ameneties_count', 'transformation_label': 'String Indexer'}], 'feature': 'Ameneties_count', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '25.74', 'stddev': '8.91', 'min': '10', 'max': '9', 'missing': '0', 'distinct': '46'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ameneties_count'}, {'feature_label': 'Ameneties_count', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ameneties_count')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'accommodates', 'transformation_label': 'String Indexer'}], 'feature': 'accommodates', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '4.09', 'stddev': '1.94', 'min': '1', 'max': '8', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'accommodates'}, {'feature_label': 'accommodates', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('accommodates')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'bedrooms', 'transformation_label': 'String Indexer'}], 'feature': 'bedrooms', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1.64', 'stddev': '0.86', 'min': '1', 'max': '8', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'bedrooms'}, {'feature_label': 'bedrooms', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('bedrooms')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'instant_bookable', 'transformation_label': 'String Indexer'}], 'feature': 'instant_bookable', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'instant_bookable'}, {'feature_label': 'instant_bookable', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('instant_bookable')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SUM', 'transformation_label': 'String Indexer'}], 'feature': 'SUM', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '149.63', 'stddev': '15.75', 'min': '104', 'max': '96', 'missing': '0', 'distinct': '48'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SUM'}, {'feature_label': 'SUM', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SUM')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Years until 2021', 'transformation_label': 'String Indexer'}], 'feature': 'Years until 2021', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5.46', 'stddev': '2.2', 'min': '1', 'max': '9', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Years until 2021'}, {'feature_label': 'Years until 2021', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Years until 2021')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Recommendation factor', 'transformation_label': 'String Indexer'}], 'feature': 'Recommendation factor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '691.79', 'stddev': '347.75', 'min': '1001', 'max': '994', 'missing': '0', 'distinct': '185'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Recommendation factor'}, {'feature_label': 'Recommendation factor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Recommendation factor')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run citiesHooks.ipynb
try:
	#sourcePreExecutionHook()

	citiesvcsv = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/5Cities_V2_CSV.csv', 'filename': '5Cities_V2_CSV.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(citiesvcsv)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run citiesHooks.ipynb
try:
	#transformationPreExecutionHook()

	citiesautofe = TransformationMain.run(citiesvcsv,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "listing_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "26789909.78", "stddev": "14356271.67", "min": "363370", "max": "48030159", "missing": "0"}, "updatedLabel": "listing_id"}, {"transformationsData": [{"feature_label": "name", "transformation_label": "String Indexer"}], "feature": "name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"\"\"Elio Suite\"\"  Vatican - Long term Available\"", "max": "\u00e6\u2014\u00a0\u00e6\u2022\u0152\u00e5\u00a4\u00a9\u00e6\u2122\u00af52\u00e5\u00b1\u201asky bar \u00e6\u203a\u00bc\u00e8\u00b0\u00b7\u00e8\u00b1\u00aa\u00e5\u008d\u017d\u00e5\u00a4\u00a7\u00e5\u00ba\u0160\u00e6\u02c6\u00bfcircle 2\u00ef\u00bc\u0152\u00e6\u2014\u00a0\u00e8\u00be\u00b9\u00e6\u00b3\u00b3\u00e6\u00b1\u00a0\u00ef\u00bc\u0152\u00e5\u0081\u00a5\u00e8\u00ba\u00ab\u00e6\u00a1\u2018\u00e6\u2039\u00bf\u00ef\u00bc\u0152\u00e4\u00ba\u00a4\u00e9\u20ac\u0161\u00e4\u00be\u00bf\u00e5\u02c6\u00a9\u00ef\u00bc\u0152\u00e6\u02dc\u017d\u00e6\u02dc\u0178\u00e5\u2026\u00ac\u00e5\u00af\u201c", "missing": "0", "distinct": "499"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "name"}, {"transformationsData": [{"feature_label": "host_id", "transformation_label": "String Indexer"}], "feature": "host_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "121441927.14", "stddev": "112299457.22", "min": "100552972", "max": "99487919", "missing": "0", "distinct": "393"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_id"}, {"transformationsData": [{"feature_label": "host_since", "transformation_label": "String Indexer"}], "feature": "host_since", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2013", "max": "9/9/2020", "missing": "0", "distinct": "429"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_since"}, {"transformationsData": [{"feature_label": "host_response_time", "transformation_label": "String Indexer"}], "feature": "host_response_time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "a few days or more", "max": "within an hour", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_response_time"}, {"transformationsData": [{"feature_label": "host_response_rate", "transformation_label": "String Indexer"}], "feature": "host_response_rate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.79", "stddev": "0.31", "min": "0", "max": "1", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_response_rate"}, {"transformationsData": [{"feature_label": "host_acceptance_rate", "transformation_label": "String Indexer"}], "feature": "host_acceptance_rate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.84", "stddev": "0.25", "min": "0", "max": "1", "missing": "0", "distinct": "52"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_acceptance_rate"}, {"transformationsData": [{"feature_label": "host_is_superhost", "transformation_label": "String Indexer"}], "feature": "host_is_superhost", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_is_superhost"}, {"transformationsData": [{"feature_label": "host_total_listings_count", "transformation_label": "String Indexer"}], "feature": "host_total_listings_count", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_total_listings_count"}, {"transformationsData": [{"feature_label": "host_has_profile_pic", "transformation_label": "String Indexer"}], "feature": "host_has_profile_pic", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_has_profile_pic"}, {"transformationsData": [{"feature_label": "host_identity_verified", "transformation_label": "String Indexer"}], "feature": "host_identity_verified", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_identity_verified"}, {"transformationsData": [{"feature_label": "neighbourhood", "transformation_label": "String Indexer"}], "feature": "neighbourhood", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Adalar", "max": "Yuen Long", "missing": "0", "distinct": "148"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "neighbourhood"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bangkok", "max": "Sydney", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "latitude", "transformation_label": "String Indexer"}], "feature": "latitude", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "8.5", "stddev": "31.27", "min": "-22.87791", "max": "48.89594", "missing": "0", "distinct": "489"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "latitude"}, {"transformationsData": [{"feature_label": "longitude", "transformation_label": "String Indexer"}], "feature": "longitude", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "13.17", "stddev": "77.78", "min": "-43.16027", "max": "29.30742", "missing": "0", "distinct": "489"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "longitude"}, {"transformationsData": [{"feature_label": "property_type", "transformation_label": "String Indexer"}], "feature": "property_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Entire bungalow", "max": "Shared room in house", "missing": "0", "distinct": "38"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "property_type"}, {"transformationsData": [{"feature_label": "room_type", "transformation_label": "String Indexer"}], "feature": "room_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Entire place", "max": "Shared room", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "room_type"}, {"transformationsData": [{"feature_label": "Ameneties_count", "transformation_label": "String Indexer"}], "feature": "Ameneties_count", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "25.74", "stddev": "8.91", "min": "10", "max": "9", "missing": "0", "distinct": "46"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ameneties_count"}, {"transformationsData": [{"feature_label": "accommodates", "transformation_label": "String Indexer"}], "feature": "accommodates", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "4.09", "stddev": "1.94", "min": "1", "max": "8", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "accommodates"}, {"transformationsData": [{"feature_label": "bedrooms", "transformation_label": "String Indexer"}], "feature": "bedrooms", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1.64", "stddev": "0.86", "min": "1", "max": "8", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "bedrooms"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "511.8", "stddev": "720.63", "min": "25", "max": "9000", "missing": "0"}, "updatedLabel": "price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "minimum_nights", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "21.16", "stddev": "50.91", "min": "3", "max": "600", "missing": "0"}, "updatedLabel": "minimum_nights"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "maximum_nights", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "110.84", "stddev": "180.49", "min": "1", "max": "1124", "missing": "0"}, "updatedLabel": "maximum_nights"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_rating", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "95.33", "stddev": "4.97", "min": "60", "max": "100", "missing": "0"}, "updatedLabel": "review_scores_rating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_accuracy", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.1", "stddev": "1.0", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "review_scores_accuracy"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_cleanliness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.21", "stddev": "1.26", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "review_scores_cleanliness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_checkin", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.41", "stddev": "0.81", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "review_scores_checkin"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_communication", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.41", "stddev": "0.88", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "review_scores_communicati..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_location", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.53", "stddev": "1.02", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "review_scores_location"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_value", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.31", "stddev": "1.09", "min": "2", "max": "10", "missing": "0"}, "updatedLabel": "review_scores_value"}, {"transformationsData": [{"feature_label": "instant_bookable", "transformation_label": "String Indexer"}], "feature": "instant_bookable", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "instant_bookable"}, {"transformationsData": [{"feature_label": "SUM", "transformation_label": "String Indexer"}], "feature": "SUM", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "149.63", "stddev": "15.75", "min": "104", "max": "96", "missing": "0", "distinct": "48"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SUM"}, {"transformationsData": [{"feature_label": "Years until 2021", "transformation_label": "String Indexer"}], "feature": "Years until 2021", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5.46", "stddev": "2.2", "min": "1", "max": "9", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Years until 2021"}, {"transformationsData": [{"feature_label": "Recommendation factor", "transformation_label": "String Indexer"}], "feature": "Recommendation factor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "691.79", "stddev": "347.75", "min": "1001", "max": "994", "missing": "0", "distinct": "185"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Recommendation factor"}]}))

	#transformationPostExecutionHook(citiesautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run citiesHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(citiesautofe, ["listing_id", "price", "minimum_nights", "maximum_nights", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "name_stringindexer", "host_id_stringindexer", "host_since_stringindexer", "host_response_time_stringindexer", "host_response_rate_stringindexer", "host_acceptance_rate_stringindexer", "host_is_superhost_stringindexer", "host_total_listings_count_stringindexer", "host_has_profile_pic_stringindexer", "host_identity_verified_stringindexer", "neighbourhood_stringindexer", "city_stringindexer", "latitude_stringindexer", "longitude_stringindexer", "property_type_stringindexer", "room_type_stringindexer", "Ameneties_count_stringindexer", "accommodates_stringindexer", "bedrooms_stringindexer", "instant_bookable_stringindexer", "SUM_stringindexer", "Years until 2021_stringindexer", "Recommendation factor_stringindexer"], "review_scores_rating")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

